In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    ManagedOnlineEndpoint,
    Model,
    CodeConfiguration,
    Environment,
)

import random

In [ ]:
# Create handle to ml workspace
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

In [ ]:
rand = random.randint(0, 10000)
endpoint_name = f"endpt-moe-{rand}"
print(endpoint_name)

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")

In [ ]:
# create endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
# print endpoint identity details
## endpoint's identity is already given blob reader permission on the default storage account
endpoint = ml_client.online_endpoints.get(endpoint_name)
print(endpoint.identity.type)
print(endpoint.identity.principal_id)

## By default Managed Endpoint identity is assigned ACR Pull role on ACR and Blob Reader role on Storage account.

In [ ]:
# read env details
import os
from dotenv import load_dotenv

load_dotenv()

## Make sure local .env.example file is renamed to .env.
## On the .env file, supply SP client id, client secret, tenant id details.

In [ ]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="./model/"),
    code_configuration=CodeConfiguration(
        code="./onlinescoring/", 
        scoring_script="score_sp.py"
    ),
    environment=Environment(
        conda_file="./environment/conda-managedidentity.yml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
    environment_variables={
        "AZURE_CLIENT_ID": os.environ["AZURE_CLIENT_ID"],
        "AZURE_TENANT_ID": os.environ["AZURE_TENANT_ID"],
        "AZURE_CLIENT_SECRET": os.environ["AZURE_CLIENT_SECRET"]
    },
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
# set traffic to 100% for deployment
endpoint.traffic = {str(deployment.name): 100}
ml_client.begin_create_or_update(endpoint).result()

In [ ]:
# test endpoint
sample_data = "./invoke/sample-request.json"
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, request_file=sample_data)